# Índice invertido en RI

Utilizar una matriz de incidencia para realizar búsqueda y recuperación de información en una colección tiene el incoveniente de requerir mayor memoria cuando la colección es muy grande. Para lidiar con esto, se propone eliminar los 0s, obteniendo un diccionario más eficiente de términos y documentos.

El <b>índice ivertido</b> asocia a cada término los documentos en los que aparece a partir de un identificador de documento. Esto evita utilizar 0s y hace más eficientes las búsqueda. Pero también requiere de una reformulación de los operadores booleanos.

A continuación mostramos una implementación de un modelo booleano basado en el índice invertido.

In [1]:
from glob import glob
from re import compile
from collections import defaultdict, Counter
from itertools import chain
import numpy as np
import pandas as pd

#### Función de tokenización

Definimos una función de tokenización muy simple que toma como token todos los elementos entre espacios en blanco, eliminando los signos no alfanuméricos.

In [2]:
regex = compile('[-_{}(),;:"#\/.¡!¿?·]')
def tokenize(text):
    """
    Función de tokenización.
    
    Arguments
    ---------
    text : str
        Cadena de texto que se tokenizará
        
    Returns
    -------
    tokens : list
        Lista de tokens
    """
    #Pasa a minúsculas
    lower_text = text.strip().lower()
    #Elimina símbolos no alfanuméricos
    alphanumeric = regex.sub('', lower_text)
    #Obtiene tokens por espacios en blanco
    tokens = alphanumeric.split()
    
    return tokens

### Intersección para operador AND

En la búsqueda booleana con listas de <i>posting</i>, los operadores booleanos deben aplicarse sobre conjunto. En el caso de AND este corresponde a una instersección de los conjuntos. Por tanto definimos una función que permita intersectar dos conjuntos. 

Esta función la ocuparemos en el modelo a partir de realizar búsquedas ordenadas para hacer más eficientes las búsquedas con operadores AND.

In [3]:
def intersect(a,b):
    """
    Función de intersección entre dos conjuntos.
    
    Arguments
    ---------
    a,b : array
        Conjuntos que se van a intersectar.
        
    Returns
    -------
        Conjunto que contiene los elementos que se intersectan.
    """
    #Guarda los resultados
    result = []
    #Inicia los índices
    i,j = 0,0
    #Recorre los conjuntos
    while i < len(a) and j < len(b):
        if a[i] == b[j]:
            #Si contiene los mismos elementos
            #agrega a la lista de intersecciones
            result.append(a[i])
            i += 1
            j += 1
            
        elif a[i] < b[j]:
            #Avanza en la primera lista
            i += 1
            
        else:
            #Avanza en la segunda lsita
            j += 1
            
    return result

## Modelo booleano con índice invertido

El modelo booleano que definimos se basa en el índice invertido. Dada una colección (guardada en un directori) el modelo crea un diccionario de términos en que cada término está asociado a una lista de posting; esta lista contiene los identificadores de documentos que pertenecen a los documentos en que dicho término aparece.

A partir del índice invertido podemos realizar búsquedas con operadores booleanos: OR, NOT, AND. Definimos las funciones que relizan estos operadores como operadores entre conjuntos. 

In [4]:
class BooleanModel(object):
    """
    Clase para crear modelo de recuperación booleana sobre una colección 
    de documentos.
    
    docs_idx : dict
        Dictionario que guarda los documentos y sus índices
    terms : list
        Lista de términos
    documentos : list
        Lista de documentos
    collection : dict
        Diccionario de índices de documentos y su lista de tokens
    incidence_matriz : array
        Matriz de incidencia término documento
    """
    def __init__(self):
        self.docIDs = {}
        self.terms = []
        self.documents = []
        self.tokens = {}
        self.dictionary = defaultdict(list)
        self.terms_frequency = None
    
    def get_documents(self,directory):
        """
        Función para obtener la colección de documentos a partir de un directorio con archivos.
        
        Arguments
        ---------
        directory : str
            Directorio donde se encuentran guardados los documentos
        """
        #Inicializa índicex
        docID = 0
        tokens = {}
        for filename in glob(directory+'*'):
            text = open(filename,'r').read()
            tokenized_text = tokenize(text)
            self.docIDs[docID] = filename
            sorted_types = list(sorted(set(tokenized_text)))
            self.tokens[docID] = sorted_types
            docID += 1
            
        #Crea la lsita de términos
        self.terms = list(set(chain(*self.tokens.values())))
        #Crea lista de documentos
        self.documents = list(self.docIDs.values())
        #Frecuencia de términos
        self.terms_frequency = Counter(chain(*self.tokens.values()))
        
    def build_inverse_index(self, directory):
        """
        Función para la creación de la matriz de incidencia.
        
        Arguments
        ---------
        directory : str
            Directorio donde se encuentran guardados los documentos
            
        Returns
        -------
            Matriz de incidencia
        """
        #Crea la colección a partir de directorio
        self.get_documents(directory)
        
        for docID, term_list in self.tokens.items():
            for term in term_list:
                self.dictionary[term].append(docID) 
    
    def AND(self, terminos):
        """
        Función para intersectar (operador AND) un conjunto de posting perteneciente a una lista larga de queries.

        Arguments
        ---------
        terminos : list
            Lista de términos para recuperar.

        Returns
        """
        #Ordena los términos por su frecuencia de menor a mayor
        terms = sorted([t for t in terminos], key=lambda t: self.terms_frequency[t])
        #Posting del término con menor frecuencia
        result = self.dictionary[terms[0]]
        #El resto de los términos
        terms.remove(terms[0])

        while terms != [] and result != []:
            #Intersecta los posting de términos
            result = intersect(result, self.dictionary[terms[0]])
            #Elimina los terminos ya intersectados
            terms.remove(terms[0])

        #Regresa los documentos que contienen
        #todos los términis
        for docID in result:
            yield self.docIDs[docID]  
            
    def NOT(self, term):
        """
        Función NOT bolleana en una lista de posting.
        
        Arguments
        ---------
        term : str
            Término sobre el que se aplica el NOT.
            
        Returns
        -------
            Documentos que corresponden al operador NOT: aquellos donde no aparece el término.
        """
        for docID, doc in self.docIDs.items():
            #Compara los docIDs del término con los documentos de la colección,
            #agrega los elementos de la colección que no pertenecen al término
            if docID not in self.dictionary[term]:
                yield doc
                
    def OR(self, terminos):
        """
        Función NOT bolleana en una lista de posting.
        
        Arguments
        ---------
        terminos : list[str]
            Lista de términos con los que se hará la operación OR.
            
        Returns
        -------
            Documentos que contienen uno o más términos.
        """
        #Une los documentos de cada términos
        for docID in chain(*[self.dictionary[t] for t in terminos]):
            yield self.docIDs[docID]

### Ejemplo de búsqueda booleana con índice invertido

Podemos utilizar la clase del modelo booleano que construimos para aplicarla a una colección en concreto. Como vemos, el diccionario del índice invertido guarda una serie de índices de documentos para cada término.

In [5]:
#Directorio de la colección
directory = 'wikipedia/'

#Creación del modelo
model = BooleanModel()
#Genera el índice invertido
model.build_inverse_index(directory)

#Imprime una entrada del diccionario
print(model.dictionary['campo'])
#print(model.documents)

[2, 5, 22, 24, 28, 30, 38, 43, 51, 64, 72, 87, 91, 92, 99]


Finalmente, podemos ver cómo funciona cada uno de los operadores booleanos en este tipo de modelo con índice invertido:

In [6]:
#Operador AND para un conjunto de términos
for doc in model.AND(['campo','política','de', 'campesinos']):
    print(doc)

wikipedia/economia (1).txt


In [7]:
#Operador NOT
for doc in model.NOT('del'):
    print(doc)

wikipedia/perro (1).txt
wikipedia/religion (2).txt
wikipedia/amor (1).txt
wikipedia/cuantica (1).txt
wikipedia/deporte (1).txt
wikipedia/sociedad (5).txt
wikipedia/sacarosa (2).txt
wikipedia/coca (1).txt
wikipedia/celula (1).txt


In [8]:
#Operador OR para un conjunto de términos
for doc in model.OR(['campo','campesinos']):
    print(doc)

wikipedia/bioinfo (1).txt
wikipedia/ciencia_historia (1).txt
wikipedia/sociedad (2).txt
wikipedia/condensador (2).txt
wikipedia/politica (1).txt
wikipedia/capoeira (2).txt
wikipedia/cuantica (1).txt
wikipedia/condensador (4).txt
wikipedia/ifai (2).txt
wikipedia/mate (2).txt
wikipedia/sociedad (1).txt
wikipedia/economia (1).txt
wikipedia/economia (2).txt
wikipedia/bioinfo (2).txt
wikipedia/condensador (3).txt
wikipedia/economia (1).txt
